In [1]:
import os

from pathling import PathlingContext
from pyspark.sql.functions import lit

MIMIC_FHIR_PATH="../_data/mimic4-demo-ptl"

In [2]:
pc = PathlingContext.create()

:: loading settings :: url = jar:file:/Users/szu004/miniconda3/envs/sof_mimic/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/szu004/.ivy2/cache
The jars for the packages stored in: /Users/szu004/.ivy2/jars
au.csiro.pathling#library-runtime added as a dependency
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-774ef320-8869-4ab8-8b66-13c6e23fc981;1.0
	confs: [default]
	found au.csiro.pathling#library-runtime;7.1.0-SNAPSHOT in local-m2-cache
	found io.delta#delta-spark_2.12;3.2.0 in local-m2-cache
	found io.delta#delta-storage;3.2.0 in local-m2-cache
	found org.antlr#antlr4-runtime;4.9.3 in local-m2-cache
	found org.apache.hadoop#hadoop-aws;3.3.4 in local-m2-cache
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in local-m2-cache
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in local-m2-cache
:: resolution report :: resolve 128ms :: artifacts dl 7ms
	:: modules in use:
	au.csiro.pathling#library-runtime;7.1.0-SNAPSHOT from local-m2-cache in [default]
	co

2024-08-28 13:19:02 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-08-28 13:19:04 INFO  Version:21 - HV000001: Hibernate Validator null
2024-08-28 13:19:04 DEBUG AbstractConfigurationImpl:176 - Setting custom MessageInterpolator of type au.csiro.pathling.shaded.org.hibernate.validator.messageinterpolation.ParameterMessageInterpolator
2024-08-28 13:19:04 DEBUG ValidationXmlParser:122 - Trying to load META-INF/validation.xml for XML based Validator configuration.
2024-08-28 13:19:04 DEBUG ResourceLoaderHelper:61 - Trying to load META-INF/validation.xml via TCCL
2024-08-28 13:19:04 DEBUG ResourceLoaderHelper:67 - Trying to load META-INF/validation.xml via Hibernate Validator's class loader
2024-08-28 13:19:04 DEBUG ValidationXmlParser:129 - No META-INF/validation.xml found. Using annotation based configuration only.
2024-08-28 13:19:04 DEBUG TraversableResolvers:61 - Cannot find jakarta.persistence.Per

In [39]:
spark = pc.spark

In [3]:
data = pc.read.parquet(MIMIC_FHIR_PATH)
data.read('Patient').show()

2024-08-28 13:19:06 WARN  SparkStringUtils:72 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+--------------------+-------------+--------+----+--------------------+------+--------------------+-------+------+----------+---------------+----------------+-------+--------------------+--------------------+--------------------+-----+-------+--------------------+-------------------+--------------------+----+----------+--------------------+
|                  id|        id_versioned|                meta|implicitRules|language|text|          identifier|active|                name|telecom|gender| birthDate|deceasedBoolean|deceasedDateTime|address|       maritalStatus|multipleBirthBoolean|multipleBirthInteger|photo|contact|       communication|generalPractitioner|managingOrganization|link|      _fid|          _extension|
+--------------------+--------------------+--------------------+-------------+--------+----+--------------------+------+--------------------+-------+------+----------+---------------+----------------+-------+--------------------+-

In [4]:
ptn_view = data.view('Patient', 
    select = [
        {
            "column": [
                {
                    "name": "id",
                    "path": "id",
                    "collection": False
                },
                {
                    "name": "gender",
                    "path": "gender",
                    "collection": False
                },
                {
                    "name": "race",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-race').extension('text').value.ofType(string)",
                    "collection": False
                },
                {
                    "name": "ethnicity",
                    "path": "extension('http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity').extension('text').value.ofType(string)",
                    "collection": False
                },
                
            ]
        }
    ]
)

ptn_view.show()

+--------------------+------+--------------------+--------------------+
|                  id|gender|                race|           ethnicity|
+--------------------+------+--------------------+--------------------+
|28dcf33b-0c52-587...|female|               White|Not Hispanic or L...|
|74a2fd87-885b-5ec...|female|               White|Not Hispanic or L...|
|51d2190c-cc46-56c...|  male|               White|  Hispanic or Latino|
|c4c29979-f2f5-56d...|  male|               White|  Hispanic or Latino|
|b410dd44-7d65-56f...|  male|               White|Not Hispanic or L...|
|a3a12d01-dc21-565...|  male|               White|Not Hispanic or L...|
|807d2a03-9c57-55e...|female|             unknown|                NULL|
|77e10fd0-6a1c-554...|female|Black or African ...|Not Hispanic or L...|
|aeda7084-be35-510...|female|               White|  Hispanic or Latino|
|5f3dcdb5-bd27-58f...|  male|               White|Not Hispanic or L...|
|2955c958-192a-50e...|female|Black or African ...|Not Hispanic o

In [5]:
ptn_view.select('race').distinct().show()

+--------------------+
|                race|
+--------------------+
|   asked but unknown|
|             unknown|
|               White|
|Black or African ...|
+--------------------+



In [6]:
data.read('Observation').limit(100).toPandas()

,id,id_versioned,meta,implicitRules,language,text,identifier,basedOn,partOf,status,...,bodySite,method,specimen,device,referenceRange,hasMember,derivedFrom,component,_fid,_extension
0,98be65be-39ab-5c6e-87fc-ea673b9a8875,Observation/98be65be-39ab-5c6e-87fc-ea673b9a8875,"(None, None, None, None, None, [http://mimic.m...",None,None,None,None,None,None,final,...,None,None,None,None,None,None,None,None,119610501,{}
1,5b5f2240-4695-5091-b52c-f45a53a0be72,Observation/5b5f2240-4695-5091-b52c-f45a53a0be72,"(None, None, None, None, None, [http://mimic.m...",None,None,None,None,None,None,final,...,None,None,None,None,None,None,None,None,264267728,{}
2,d1dde4fa-799a-5c2a-ba7e-01944056c18a,Observation/d1dde4fa-799a-5c2a-ba7e-01944056c18a,"(None, None, None, None, None, [http://mimic.m...",None,None,None,None,None,None,final,...,None,None,None,None,None,None,None,None,1068373597,{}
3,61c37916-95fe-59d1-b75f-1766aaaec21e,Observation/61c37916-95fe-59d1-b75f-1766aaaec21e,"(None, None, None, None, None, [http://mimic.m...",None,None,None,None,None,None,final,...,None,None,None,None,None,None,None,None,1369831003,{}
4,4ce17837-671c-5124-9b96-cd0be8f08912,Observation/4ce17837-671c-5124-9b96-cd0be8f08912,"(None, None, None, None, None, [http://mimic.m...",None,None,None,None,None,None,final,...,None,None,None,None,None,None,None,None,1056796779,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,23dc81b2-d9fa-5910-a22a-f1a97b9c9edf,Observation/23dc81b2-d9fa-5910-a22a-f1a97b9c9edf,"(None, None, None, None, None, [http://mimic.m...",None,None,None,None,None,None,final,...,None,None,None,None,None,None,None,None,629644022,{}
96,2abc0bbf-1213-5da9-9d77-9251d4bc9337,Observation/2abc0bbf-1213-5da9-9d77-9251d4bc9337,"(None, None, None, None, None, [http://mimic.m...",None,None,None,None,None,None,final,...,None,None,None,None,None,None,None,None,561159824,{}
97,8559a53e-42b8-5a0c-90ff-08001336d1e4,Observation/8559a53e-42b8-5a0c-90ff-08001336d1e4,"(None, None, None, None, None, [http://mimic.m...",None,None,None,None,None,None,final,...,None,None,None,None,None,None,None,None,1785813077,{}
98,939bc6ee-7008-5ba4-8b9b-573862b041f3,Observation/939bc6ee-7008-5ba4-8b9b-573862b041f3,"(None, None, None, None, None, [http://mimic.m...",None,None,None,None,None,None,final,...,None,None,None,None,None,None,None,None,2126783329,{}


## Observations 
### Chartevents

In [57]:
def obs_chartevent_for_code(code, view_name = None):
    obs_view = data.view('Observation',
                          select = [
                              {
                                  "column": [
                                      {
                                          "name": "id",
                                          "path": "id",
                                          "type": "id",
                                          "collection": False
                                      },
                                      {
                                          "name": "patient_id",
                                          "path": "subject.getReferenceKey()",
                                          "type": "string",
                                          "collection": False
                                      },
                                      {
                                          "name": "encounter_id",
                                          "path": "encounter.getReferenceKey()",
                                          "type": "string",
                                          "collection": False
                                      },
                                      {
                                          "name": "date",
                                          "path": "effectiveDateTime",
                                          "type": "dateTime",
                                          "collection": False
                                      },
                                      {
                                          "name": "value",
                                          "path": "valueQuantity.value",
                                          "type": "decimal",
                                          "collection": False
                                      },
                                      {
                                          "name": "unit",
                                          "path": "valueQuantity.unit",
                                          "collection": False
                                      }
                                  ]
                              }
                          ],
                          where= [
                              {
                                  "path":"code.coding.system = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-chartevents-d-items'"
                              },
                              {
                                  "path":f"code.coding.code = '{code}'"
                              },

                          ]
    )
    if view_name is not None:
        obs_view.createOrReplaceTempView(view_name)
    return obs_view

In [58]:
obs_resp_rate = obs_chartevent_for_code('224690', 'obs_resp_rate')
obs_resp_rate.show()

+--------------------+--------------------+--------------------+--------------------+---------+--------+
|                  id|          patient_id|        encounter_id|                date|    value|    unit|
+--------------------+--------------------+--------------------+--------------------+---------+--------+
|fcae13c7-ddd9-5d4...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T00:00:...|18.000000|insp/min|
|a3c20e0d-9c7b-5a6...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T07:00:...|14.000000|insp/min|
|b22b0632-3352-505...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T11:00:...|14.000000|insp/min|
|5c457b43-a9fc-5f7...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T15:00:...|14.000000|insp/min|
|f2b42818-fa58-5fb...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T20:00:...|16.000000|insp/min|
|1ba6f7af-a172-54c...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-25T00:00:...|14.000000|insp/min|
|bdd48277-3cea-53b...|Patient/2955c958-...|Encounter/84

In [59]:
obs_spo2 = obs_chartevent_for_code('20277', 'obs_spo2')
obs_spo2.show()

+---+----------+------------+----+-----+----+
| id|patient_id|encounter_id|date|value|unit|
+---+----------+------------+----+-----+----+
+---+----------+------------+----+-----+----+



In [60]:
obs_heart_rate = obs_chartevent_for_code('220045', 'obs_heart_rate')
obs_heart_rate.show()

+--------------------+--------------------+--------------------+--------------------+----------+----+
|                  id|          patient_id|        encounter_id|                date|     value|unit|
+--------------------+--------------------+--------------------+--------------------+----------+----+
|d35bbfbf-73bf-5b5...|Patient/2955c958-...|Encounter/2b6a330...|2178-07-05T07:42:...|103.000000| bpm|
|072fd1e4-6b01-517...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T00:00:...| 84.000000| bpm|
|a30dce5c-48ae-5a3...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T01:00:...| 87.000000| bpm|
|121ee7e1-2ef1-5cd...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T02:00:...| 89.000000| bpm|
|a1313740-5fc4-533...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T03:00:...| 81.000000| bpm|
|211e2401-d817-5d1...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T04:00:...| 87.000000| bpm|
|d36d5b72-7dcb-536...|Patient/2955c958-...|Encounter/84bdbd6...|2178-07-24T05:00:.

In [64]:
#
# alike mimiciv_derrived.vital_signs
#
derr_vital_signs_view = spark.sql("""
WITH ou AS (
SELECT patient_id, encounter_id, date, value AS resp_rate, NULL as heart_rate FROM obs_resp_rate 
UNION  
SELECT patient_id, encounter_id, date, NULL AS resp_rate, value as heart_rate FROM obs_heart_rate 
) 
SELECT patient_id, encounter_id, date, FIRST(resp_rate) AS resp_rate, FIRST(heart_rate) AS heart_rate FROM ou 
GROUP BY patient_id, encounter_id, date
""")
derr_vital_signs_view.sample(0.02).toPandas()

,patient_id,encounter_id,date,resp_rate,heart_rate
0,Patient/a2605b15-4f1b-5839-b4ce-fb7a6bc1005f,Encounter/5041d382-c2f6-570f-a28b-9511135d47bf,2135-01-16T23:00:00+10:00,20.000000,None
1,Patient/4365e125-c049-525a-9459-16d5e6947ad2,Encounter/b742bb23-5e32-5ae2-ba52-37e21e070765,2116-12-06T04:00:00+10:00,12.000000,None
2,Patient/a7bbf9a2-f7fe-5815-a637-fa59bd70b374,Encounter/92d3de1d-c030-54f0-8023-f73c91a8e2b7,2111-11-14T00:19:00+10:00,29.000000,None
3,Patient/a2605b15-4f1b-5839-b4ce-fb7a6bc1005f,Encounter/5041d382-c2f6-570f-a28b-9511135d47bf,2135-01-11T15:00:00+10:00,16.000000,None
4,Patient/a2605b15-4f1b-5839-b4ce-fb7a6bc1005f,Encounter/5041d382-c2f6-570f-a28b-9511135d47bf,2135-01-11T03:00:00+10:00,20.000000,None
...,...,...,...,...,...
273,Patient/f77a5b72-65fd-5b20-8cef-6b6be4791265,Encounter/b34e83f0-863c-533f-a5c0-84fcb98c20bb,2176-11-27T21:00:00+10:00,None,73.000000
274,Patient/f5efdf3f-5b53-5c9f-95a6-047275107c46,Encounter/25e05468-7cbf-5a04-9209-79cb07703326,2141-05-25T22:00:00+10:00,None,102.000000
275,Patient/e2beb281-c44f-579b-8211-a3749c549e92,Encounter/063983ba-71d7-5e3e-8f64-08433c20e377,2201-12-13T03:00:00+10:00,None,82.000000
276,Patient/f572075b-186b-5565-b2d4-1567fe4f925c,Encounter/88df0619-e6cd-5900-bbdc-ba5b241fbda8,2125-09-29T12:00:00+10:00,None,81.000000


In [74]:
spark.sql("""
SELECT patient_id, encounter_id, date, CAST(date AS TIMESTAMP) AS datetime, value AS resp_rate, NULL as heart_rate FROM obs_resp_rate
""").limit(10).toPandas()

,patient_id,encounter_id,date,datetime,resp_rate,heart_rate
0,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/84bdbd69-441b-5264-93de-cbbda8d11822,2178-07-24T00:00:00+10:00,2178-07-24 00:00:00,18.000000,None
1,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/84bdbd69-441b-5264-93de-cbbda8d11822,2178-07-24T07:00:00+10:00,2178-07-24 07:00:00,14.000000,None
2,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/84bdbd69-441b-5264-93de-cbbda8d11822,2178-07-24T11:00:00+10:00,2178-07-24 11:00:00,14.000000,None
3,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/84bdbd69-441b-5264-93de-cbbda8d11822,2178-07-24T15:00:00+10:00,2178-07-24 15:00:00,14.000000,None
4,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/84bdbd69-441b-5264-93de-cbbda8d11822,2178-07-24T20:00:00+10:00,2178-07-24 20:00:00,16.000000,None
5,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/84bdbd69-441b-5264-93de-cbbda8d11822,2178-07-25T00:00:00+10:00,2178-07-25 00:00:00,14.000000,None
6,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/84bdbd69-441b-5264-93de-cbbda8d11822,2178-07-25T08:00:00+10:00,2178-07-25 08:00:00,14.000000,None
7,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/84bdbd69-441b-5264-93de-cbbda8d11822,2178-07-25T11:00:00+10:00,2178-07-25 11:00:00,14.000000,None
8,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/84bdbd69-441b-5264-93de-cbbda8d11822,2178-07-22T08:00:00+10:00,2178-07-22 08:00:00,17.000000,None
9,Patient/2955c958-192a-50eb-b59d-23a29d7d374e,Encounter/84bdbd69-441b-5264-93de-cbbda8d11822,2178-07-22T12:00:00+10:00,2178-07-22 12:00:00,21.000000,None


### Labevents

In [87]:
def obs_labitems_for_code(code, view_name = None):
    obs_view = data.view('Observation',
                         select = [
                             {
                                 "column": [
                                     {
                                         "name": "id",
                                         "path": "id",
                                         "type": "id",
                                         "collection": False
                                     },
                                     {
                                         "name": "patient_id",
                                         "path": "subject.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "encounter_id",
                                         "path": "encounter.getReferenceKey()",
                                         "type": "string",
                                         "collection": False
                                     },
                                     {
                                         "name": "date",
                                         "path": "effectiveDateTime",
                                         "type": "dateTime",
                                         "collection": False
                                     },
                                     {
                                         "name": "value",
                                         "path": "valueQuantity.value",
                                         "type": "decimal",
                                         "collection": False
                                     },
                                     {
                                         "name": "unit",
                                         "path": "valueQuantity.unit",
                                         "collection": False
                                     }
                                 ]
                             }
                         ],
                         where= [
                             {
                                 "path":"code.coding.system = 'http://mimic.mit.edu/fhir/mimic/CodeSystem/mimic-d-labitems'"
                             },
                             {
                                 "path":f"code.coding.code = '{code}'"
                             },
                         ]
    )
    if view_name is not None:
        obs_view.createOrReplaceTempView(view_name)
    return obs_view

In [88]:
# bg so2
obs_bg_so2 = obs_labitems_for_code('50817', 'obs_bg_so2')
# AND < 100
obs_bg_so2.show()

+--------------------+--------------------+--------------------+--------------------+---------+----+
|                  id|          patient_id|        encounter_id|                date|    value|unit|
+--------------------+--------------------+--------------------+--------------------+---------+----+
|69ca4208-7476-593...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-05T22:17:...|98.000000|   %|
|34706e68-6ab5-505...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-06T05:23:...|97.000000|   %|
|64ff5dcd-f26c-556...|Patient/bc2a74ce-...|                NULL|2140-10-03T07:15:...|74.000000|   %|
|2ff0afd6-942c-579...|Patient/bc2a74ce-...|Encounter/b7566a4...|2140-10-03T09:25:...|73.000000|   %|
|0d18961c-e1db-59f...|Patient/bc2a74ce-...|                NULL|2137-06-19T15:48:...|73.000000|   %|
|550dd506-7fd2-510...|Patient/c3412a5f-...|Encounter/5ee5f4a...|2119-10-26T07:22:...|97.000000|   %|
|26725c70-7885-5c2...|Patient/c3412a5f-...|                NULL|2119-10-26T06:25:...|48.000

In [89]:
# bg so2
obs_bg_pco2 = obs_labitems_for_code('50818', 'obs_bg_pco2')
obs_bg_pco2.show()

+--------------------+--------------------+--------------------+--------------------+---------+-----+
|                  id|          patient_id|        encounter_id|                date|    value| unit|
+--------------------+--------------------+--------------------+--------------------+---------+-----+
|bbe0d98c-f367-570...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-06T00:10:...|38.000000|mm Hg|
|11f6e65e-e072-5dd...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-05T22:17:...|39.000000|mm Hg|
|023811f9-620c-545...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-06T05:23:...|35.000000|mm Hg|
|1bbf8896-ec5d-5f7...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-05T16:34:...|39.000000|mm Hg|
|19333fc0-08ad-54f...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-05T13:46:...|47.000000|mm Hg|
|fa450e6f-0ba2-529...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-05T19:49:...|50.000000|mm Hg|
|cf6412d3-def6-50a...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-06T02:53:.

In [90]:
# hemoglobin
obs_bg_hemoglobin = obs_labitems_for_code('50811', 'obs_bg_hemoglobin')
obs_bg_hemoglobin.show()

+--------------------+--------------------+--------------------+--------------------+---------+----+
|                  id|          patient_id|        encounter_id|                date|    value|unit|
+--------------------+--------------------+--------------------+--------------------+---------+----+
|1236173a-d92a-500...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-05T16:34:...|10.900000|g/dL|
|a88475dd-25e1-50b...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-05T13:46:...|14.100000|g/dL|
|4f4e7911-5303-50d...|Patient/bbad4581-...|Encounter/508ab45...|2167-05-05T17:57:...|11.000000|g/dL|
|1f6fd236-270b-528...|Patient/c0d83ad6-...|Encounter/4125afa...|2180-02-21T08:02:...|16.000000|g/dL|
|5546a723-83ce-590...|Patient/c0d83ad6-...|Encounter/4125afa...|2180-02-21T10:15:...|12.500000|g/dL|
|385ef391-873a-52b...|Patient/c0d83ad6-...|Encounter/4125afa...|2180-02-21T09:38:...|11.700000|g/dL|
|4ca95ae9-df89-523...|Patient/c3412a5f-...|                NULL|2119-10-26T06:25:...|18.400